## 拆分订单表，且关联明细

### 要求如下
* 主表：会员销售数据，从表：会员销售商品明细
* 主表长度2W
* 主表的件数、金额 = 从表的件数、金额
* 校验维度：主表的订单件数=从表的销售件数；主表的订单应付金额 = 从表的商品总金额

In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
filename_order  =  r'D:\数据项目\23-EGG\待导入\第一批可导入\订单\会员销售数据.xlsx'
filename_orderdtl  =  r'D:\数据项目\23-EGG\待导入\第一批可导入\订单\会员销售数据.xlsx'
# file = r'D:\数据项目\16-方所\待导入-第一批\订单数据\订单明细.csv'
# f = open(filename_order,encoding='utf-8')
# order = pd.read_csv(f,dtype = {'门店编码' : str,'订单编号' : str,'线下卡号' : str})

order = pd.read_excel(filename_order,sheet_name=0,dtype={'门店编码': 'str','订单编号': 'str','线下卡号': 'str'}) #'关联单号': 'str'
orderdtl = pd.read_excel(filename_orderdtl,sheet_name=1,dtype={'门店编码': 'str','订单编号': 'str','商品条码': 'str'})

# f = open(filename_orderdtl,encoding='utf-8')
# orderdtl = pd.read_csv(f,dtype = {'门店编码' : str,'订单编号' : str,'商品条码' : str})

row_num = int(order.shape[0])  # 读取主表长度


In [3]:
order.shape,orderdtl.shape

((179600, 14), (688875, 10))

In [4]:

for i in range (0,row_num,20000):
    j = i+20000 if i+20000 < row_num else  row_num
    order_cut = order[i:j]
    orderdtl_cut =orderdtl[orderdtl['订单编号'].isin(order_cut['订单编号'])]
    writer = pd.ExcelWriter("D:\\数据项目\\23-EGG\\待导入\\第一批可导入\\订单\\%s_%s"%(int(i/20000),os.path.basename(filename_order)))
    order_cut.to_excel(writer, sheet_name='会员销售数据',index = False)
    orderdtl_cut.to_excel(writer, sheet_name='会员销售商品明细',index = False)
    writer.save()
print("导出完成")

导出完成


### 若存在订单编号和门店编码为索引则执行以下方案


In [ ]:
order['索引'] = order["门店编码"].map(str) + order["订单编号"]
orderdtl['索引'] = orderdtl["门店编码"].map(str) + orderdtl["订单编号"]

In [ ]:
for i in range (0,row_num,20000):
    j = i+20000 if i+20000 < row_num else  row_num
    order_cut = order[i:j]
    orderdtl_cut =orderdtl[orderdtl['索引'].isin(order_cut['索引'])]
    
    writer = pd.ExcelWriter("C:\\Users\\Administrator\\Desktop\\拆表\\%s_%s"%(int(i/20000),os.path.basename(filename_order)))
    ordercolumns_sort=['门店编码', '订单编号', '关联单号', '订单状态', '线下卡号', '订单时间', '订单件数', '订单吊牌金额',
       '订单应付金额', '订单实付金额', '销售导购工号', '推荐导购工号', '推荐门店编码', '订单来源']
    order_cut=order_cut.reindex(columns=ordercolumns_sort)
    order_cut.to_excel(writer, sheet_name='会员销售数据',index = False)
    
    orderdtlcolumns_sort=['门店编码', '订单编号', '商品条码', '商品吊牌价', '商品实际售价', '销售件数', '商品总金额', '销售导购工号',
       '推荐导购工号', '推荐门店编码']
    orderdtl_cut = orderdtl_cut.reindex(columns=orderdtlcolumns_sort)    
    orderdtl_cut.to_excel(writer, sheet_name='会员销售商品明细',index = False)
    writer.save()
print("导出完成")

In [ ]:
orderdtl.columns

In [ ]:
order.columns